# **Código original - Modificado**

# **1. Criar a variavél de ambiente com a chave API Groq**


In [ ]:
from pathlib import Path
from dotenv import load_dotenv
import os
project_root = Path.cwd().parent.parent  # caminho da raíz do projeto

# Carregar o .env
env_path = project_root / '.env'
load_dotenv(env_path)
api_key = os.getenv('GROQ_API_KEY')


# **2. Criar conexão com a API da Groq**

In [ ]:
import os
from groq import Groq


client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[                                                              # Lista de mensagens da conversa
        {
            "role": "user",                                                 #indica que é mensagem do usuário
            "content": "Explain the importance of fast language models",    # o Texto da pergunta
        }
    ],
    model="llama-3.3-70b-versatile",                                        # Qual modelo utilizar
)

print(chat_completion.choices[0].message.content)

# choices[0]: primeira (e única) resposta gerada
# .message.content: extrai só o texto da resposta

# **3. Testar chamada de REST API EXTERNAS**

In [ ]:
# ============================================================================
# VALIDAÇÃO DE FERRAMENTAS EXTERNAS: REST Countries
# ============================================================================

import requests
from typing import Dict

# ============================================================================
# CONFIGURAÇÃO
# ============================================================================

PAIS = "Brazil"

# Coordenadas de bounding box para cidades principais
# Formato: (sul, oeste, norte, leste)

# ============================================================================
# FERRAMENTA 1: REST Countries API
# ============================================================================

def get_country_info(country_name: str) -> Dict: #função que captura os dados 
    """
    Busca informações básicas de um país
    """
    url = f"https://restcountries.com/v3.1/name/{country_name}"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()[0]

        result = {
            "country": data['name']['common'],
            "capital": data['capital'][0] if data.get('capital') else "N/A",
            "population": data['population'],
            "region": data['region'],
            "subregion": data.get('subregion', 'N/A'),
            "languages": list(data.get('languages', {}).values()),
            "currencies": {
                code: info['name'] + f" ({info.get('symbol', '')})"
                for code, info in data.get('currencies', {}).items()
            },
            "ddi": data['idd']['root'] + (data['idd'].get('suffixes', [''])[0]),
            "area_km2": data.get('area', 0),
            "timezones": data.get('timezones', []),
            "borders": data.get('borders', [])
        }
        
        return result
        
    except Exception as e:
        return {"error": str(e)}


# ============================================================================
# EXECUÇÃO DOS TESTES
# ============================================================================

print("=" * 80)
print("VALIDACAO DE FERRAMENTAS EXTERNAS")
print("=" * 80)
print(f"\nPais: {PAIS}")

# ----------------------------------------------------------------------------
# TESTE 1: REST Countries API
# ----------------------------------------------------------------------------

print("-" * 80)
print("TESTE 1: REST Countries API")
print("-" * 80)

country_data = get_country_info(PAIS)

if "error" in country_data:
    print(f"ERRO: {country_data['error']}")
else:
    print(f"Status: OK")
    print(f"\nPais: {country_data['country']}")
    print(f"Capital: {country_data['capital']}")
    print(f"Populacao: {country_data['population']:,} habitantes")
    print(f"Regiao: {country_data['region']} ({country_data['subregion']})")
    print(f"Idiomas: {', '.join(country_data['languages'])}")
    
    currencies = [f"{code}: {name}" for code, name in country_data['currencies'].items()]
    print(f"Moedas: {', '.join(currencies)}")
    
    print(f"DDI: {country_data['ddi']}")
    print(f"Area: {country_data['area_km2']:,.0f} km2")
    print(f"Fusos: {', '.join(country_data['timezones'])}")
    
    if country_data['borders']:
        print(f"Fronteiras: {', '.join(country_data['borders'])}")



In [ ]:
# ============================================================================
# TESTE SIMPLES: Valor do Real (BRL) em relação ao Dólar (USD)
# ============================================================================

import requests

# API Frankfurter - SEM API KEY!
url = "https://api.frankfurter.dev/v1/latest?base=USD&symbols=BRL"

# Fazer a requisição
response = requests.get(url)
data = response.json()

# Mostrar resultado
print("=" * 50)
print("COTACAO DO REAL")
print("=" * 50)
print(f"Data: {data['date']}")
print(f"1 USD = {data['rates']['BRL']} BRL")
print("=" * 50)

# **4. Criar a Classe do Agente**

In [ ]:
class Agent:
    def __init__(self, client, system): #Construtor, recebe os argumentos que serão os atributos do objeto
        self.client= client             #Conexão com API Groq,
        self.system= system             #Prompt que define o comportamento do agente | Define personalidade/função (ex: "Você é um assistente técnico")
        self.messages= []               #Armazenar o histórico da conversa
        if self.system is not None:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message=''): #Permite usar a classe como função
        if message:             
            self.messages.append({"role": "user", "content": message})  #Adiciona a pergunta (input) do usario,
        result= self.execute()                                          #Chamada da API do Groq
        self.messages.append({"role": "assistant", "content": result})  #Salva a resposta do modelo, resposta com base no histórico
        return result
    
    def execute(self):
        completion= client.chat.completions.create(
        messages=self.messages,                         #Envia todo o histórico
        model="llama-3.3-70b-versatile",                # Qual modelo está utilizando
    )
        return completion.choices[0].message.content

# **5. Implementando Padrão ReAct**
  
    
    1. Thought   → Raciocínio sobre o problema

    2. Action    → Executa ferramenta

    3. PAUSE     → Aguarda resultado

    4. Observation → Recebe resultado da ferramenta
    
    5. (Volta ao Thought ou dá Answer final)
    


**5.1 System Prompt**

In [ ]:
system_prompt = """You are a Country Information Assistant. You help users with questions about countries using available tools.

CRITICAL LANGUAGE RULE:
- ALWAYS call APIs with country names in ENGLISH (e.g., "Brazil", "Japan", "France", "Germany")
- Detect the language from the user's input
- Write your Thoughts and final Answer in the SAME language as the user's question
- API calls are always in English, but your reasoning and answer match the user's language

You run in a loop of Thought, Action, PAUSE, Observation.
At the end you output an Answer.

Available actions:

get_country_info:
e.g. get_country_info: Brazil
ALWAYS use English country names
Returns: capital, population, language, currency code, timezone, area

convert_currency:
e.g. convert_currency: BRL, JPY
Currency codes in ISO 4217 format
Returns: exchange rate (1 unit of first currency to second)

calculate_timezone_diff:
e.g. calculate_timezone_diff: UTC-03:00, UTC+09:00
Returns: what time it is in destination when it's 12:00 in origin

INSTRUCTIONS:
1. Detect the language from the user's question
2. Understand what the user wants to know
3. Decide which tools you need to answer the question
4. Use tools strategically - you don't need to use all tools for every question
5. Write Thoughts and Answer in the user's language
6. Be informative, clear, and concise

COMMON USE CASES:

Simple country info:
- "Tell me about Japan" → Just use get_country_info
- "What's the capital of France?" → Just use get_country_info

Currency questions:
- "How much is 1 dollar in euros?" → Use convert_currency
- "What's the exchange rate BRL to USD?" → Use convert_currency

Timezone questions:
- "What time is it in Tokyo when it's noon in Brazil?" → Use calculate_timezone_diff
- "Time difference between Paris and New York?" → Get both countries info, then calculate_timezone_diff

Country comparisons:
- "Compare Brazil and Japan" → Use get_country_info for both, convert_currency, calculate_timezone_diff
- "Brazil vs Tunisia" → Get both countries, convert currency, calculate timezone

General questions:
- Analyze what the user wants
- Use only the tools needed to answer
- Don't force all tools if not necessary

EXAMPLES:

Example 1 - Simple info (English):

Question: Tell me about Germany

Thought: User wants basic information about Germany. I'll get the country info.
Action: get_country_info: Germany
PAUSE

Observation: Country: Germany
Capital: Berlin
Population: 83,294,633
Language: German
Currency: Euro (EUR) €
Timezone: UTC+01:00
Area: 357,114 km²

Answer: Germany Information

Capital: Berlin
Population: 83.3 million people
Language: German
Currency: Euro (EUR)
Timezone: UTC+01:00
Area: 357,114 km²

Example 2 - Simple info (Portuguese):

Question: Me fale sobre a Alemanha

Thought: O usuário quer informações básicas sobre a Alemanha. Vou buscar os dados do país.
Action: get_country_info: Germany
PAUSE

Observation: Country: Germany
Capital: Berlin
Population: 83,294,633
Language: German
Currency: Euro (EUR) €
Timezone: UTC+01:00
Area: 357,114 km²

Answer: Informações sobre a Alemanha

Capital: Berlim
População: 83,3 milhões de habitantes
Idioma: Alemão
Moeda: Euro (EUR)
Fuso horário: UTC+01:00
Área: 357.114 km²

Example 3 - Currency only (English):

Question: What's the exchange rate from USD to EUR?

Thought: User wants currency conversion. I'll use convert_currency.
Action: convert_currency: USD, EUR
PAUSE

Observation: 1 USD = 0.9234 EUR

Answer: Currency Exchange Rate

1 US Dollar = 0.92 Euro

Example 4 - Timezone only (Portuguese):

Question: Que horas são no Japão quando são 15h no Brasil?

Thought: Usuário quer saber diferença de horário. Primeiro preciso dos fusos horários dos dois países.
Action: get_country_info: Brazil
PAUSE

Observation: Country: Brazil
Timezone: UTC-03:00

Thought: Agora preciso do fuso do Japão
Action: get_country_info: Japan
PAUSE

Observation: Country: Japan
Timezone: UTC+09:00

Thought: Agora vou calcular, mas a ferramenta usa 12h como base. Preciso adaptar mentalmente.
Action: calculate_timezone_diff: UTC-03:00, UTC+09:00
PAUSE

Observation: 12:00 in UTC-03:00 = 00:00 in UTC+09:00 (next day)

Answer: Diferença de Horário

O Japão está 12 horas à frente do Brasil.
Quando são 15h (3 da tarde) no Brasil, são 3h (3 da manhã do dia seguinte) no Japão.

Example 5 - Full comparison (Portuguese):

Question: Estou no Brasil, me fale sobre a Tunísia

Thought: Usuário quer informações sobre a Tunísia, provavelmente comparando com o Brasil. Vou buscar informações de ambos os países.
Action: get_country_info: Brazil
PAUSE

Observation: Country: Brazil
Capital: Brasília
Population: 213,421,037
Currency: Brazilian real (BRL)
Timezone: UTC-03:00
Area: 8,515,767 km²

Thought: Agora vou buscar informações da Tunísia
Action: get_country_info: Tunisia
PAUSE

Observation: Country: Tunisia
Capital: Tunis
Population: 11,972,169
Currency: Tunisian dinar (TND)
Timezone: UTC+01:00
Area: 163,610 km²

Thought: Vou converter a moeda
Action: convert_currency: BRL, TND
PAUSE

Observation: 1 BRL = 0.5432 TND

Thought: Agora calcular o fuso horário
Action: calculate_timezone_diff: UTC-03:00, UTC+01:00
PAUSE

Observation: 12:00 in UTC-03:00 = 16:00 in UTC+01:00

Answer: Comparação Brasil vs Tunísia

Moeda: 1 Real Brasileiro = 0,54 Dinares Tunisianos
Horário: Quando são 12h no Brasil, são 16h na Tunísia (4 horas à frente)
Comparação:
   - Brasil é 52 vezes maior em área (8,5M km² vs 164K km²)
   - Brasil tem 17,8 vezes mais habitantes (213M vs 12M)
   - Capitais: Brasília vs Túnis
   - Idiomas: Português vs Árabe

IMPORTANT NOTES:
- If a tool fails, acknowledge it gracefully and provide other available information
- Don't use tools unnecessarily - if the user asks only about one country, don't fetch two
- Be flexible - adapt your approach based on what the user actually wants to know
- Always respect the user's language for communication

Now it's your turn:
""".strip()


**5.2 Ferramentas utilizadas**

In [ ]:
def get_country_info(country_name):
    """
    Busca informações completas de um país via REST Countries API
    
    Args:
        country_name (str): Nome do país em inglês (ex: "Brazil", "Japan", "France")
    
    Returns:
        str: Informações formatadas do país ou mensagem de erro
    
    Exemplo:
        >>> get_country_info("Brazil")
        Country: Brazil
        Capital: Brasília
        Population: 213,421,037
        Language: Portuguese
        Currency: Brazilian real (BRL) R$
        Timezone: UTC-03:00
        Area: 8,515,767 km²
    """
    import requests
    
    url = f"https://restcountries.com/v3.1/name/{country_name.strip()}"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()[0]
        
        # Extrair dados principais
        capital = data['capital'][0] if data.get('capital') else "N/A"
        population = f"{data['population']:,}"
        languages = ", ".join(data.get('languages', {}).values())
        
        # Processar moeda
        currencies = data.get('currencies', {})
        if currencies:
            curr_code = list(currencies.keys())[0]
            curr_info = currencies[curr_code]
            currency = f"{curr_info['name']} ({curr_code})"
            if 'symbol' in curr_info:
                currency += f" {curr_info['symbol']}"
        else:
            currency = "N/A"
        
        # Timezone principal (primeiro da lista, geralmente da capital)
        timezone = data.get('timezones', ['UTC'])[0]
        
        # Área
        area = f"{data.get('area', 0):,.0f}"
        
        # Retornar formatado
        return f"""Country: {data['name']['common']}
Capital: {capital}
Population: {population}
Language: {languages}
Currency: {currency}
Timezone: {timezone}
Area: {area} km²"""
        
    except Exception as e:
        return f"ERROR: Could not fetch data for {country_name}"

In [ ]:
def convert_currency(from_code, to_code):
    """
    Converte 1 unidade da moeda origem para moeda destino usando Frankfurter API
    
    Args:
        from_code (str): Código ISO 4217 da moeda origem (ex: "BRL", "USD", "EUR")
        to_code (str): Código ISO 4217 da moeda destino (ex: "JPY", "GBP", "CNY")
    
    Returns:
        str: Taxa de conversão formatada ou mensagem de erro
    
    Exemplo:
        >>> convert_currency("BRL", "USD")
        1 BRL = 0.1723 USD
        
        >>> convert_currency("EUR", "JPY")
        1 EUR = 163.2547 JPY
    """
    import requests
    
    from_code = from_code.strip().upper()
    to_code = to_code.strip().upper()
    
    url = f"https://api.frankfurter.dev/v1/latest?base={from_code}&symbols={to_code}"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        rate = data['rates'][to_code]
        
        return f"1 {from_code} = {rate:.4f} {to_code}"
        
    except Exception as e:
        return f"ERROR: Could not convert {from_code} to {to_code}"

In [ ]:
def calculate_timezone_diff(tz_origin, tz_destination):
    """
    Calcula que horas são no destino quando são 12:00 na origem
    
    Args:
        tz_origin (str): Timezone do país origem (ex: "UTC-03:00", "UTC+05:30")
        tz_destination (str): Timezone do país destino (ex: "UTC+09:00")
    
    Returns:
        str: Horário correspondente no destino ou mensagem de erro
    
    Exemplo:
        >>> calculate_timezone_diff("UTC-03:00", "UTC+09:00")
        12:00 in UTC-03:00 = 00:00 in UTC+09:00 (next day)
        
        >>> calculate_timezone_diff("UTC+00:00", "UTC-05:00")
        12:00 in UTC+00:00 = 07:00 in UTC-05:00
    """
    import re
    
    try:
        def extract_offset(tz):
            """Extrai offset em horas de uma string timezone"""
            if tz == "UTC" or tz == "UTC+00:00":
                return 0
            
            match = re.search(r'UTC([+-])(\d{1,2}):?(\d{2})?', tz)
            if match:
                sign = 1 if match.group(1) == '+' else -1
                hours = int(match.group(2))
                minutes = int(match.group(3) or 0)
                return sign * (hours + minutes / 60)
            return 0
        
        offset_origin = extract_offset(tz_origin)
        offset_dest = extract_offset(tz_destination)
        diff = offset_dest - offset_origin
        
        # Calcular horário no destino (assumindo 12:00 na origem)
        hour_dest = (12 + diff) % 24
        
        # Determinar se é dia seguinte ou anterior
        if diff >= 12:
            day_note = " (next day)"
        elif diff <= -12:
            day_note = " (previous day)"
        else:
            day_note = ""
        
        return f"12:00 in {tz_origin} = {int(hour_dest):02d}:00 in {tz_destination}{day_note}"
        
    except Exception as e:
        return f"ERROR: Could not calculate timezone difference"

# 6. Implementando looping automático

In [ ]:
import re
from datetime import datetime


def agent_loop(max_iterations, system, current_country, destination_country, query="", 
               log_file="agent_log.txt"):
    """
    Loop principal do agente de comparação de países com log em arquivo
    
    Args:
        max_iterations (int): Limite máximo de iterações
        system (str): System prompt que define o comportamento do agente
        query (str): Query inicial (será formatada automaticamente)
        current_country (str): País atual do usuário (ex: "Brazil")
        destination_country (str): País de destino (ex: "Japan")
        log_file (str): Nome do arquivo para salvar o log completo
    
    Returns:
        str: Resposta final do agente
    """
    
    # Criar o agente com a conexão e system prompt
    agent = Agent(client, system)
    
    # Lista de ferramentas disponíveis para o agente
    tools = ['get_country_info', 'convert_currency', 'calculate_timezone_diff']
    
    if query:
        next_prompt= query
    else:    
        # Formatar query inicial com os países informados
        next_prompt = f"I'm in {current_country}, tell me about {destination_country}"
    
    # Contador de iterações
    i = 0
    
    # Variável para armazenar resultado final
    final_result = ""
    
    # Abrir arquivo de log para escrita
    with open(log_file, 'w', encoding='utf-8') as f:
        # Cabeçalho do log
        f.write("=" * 80 + "\n")
        f.write("LOG COMPLETO DO AGENTE DE COMPARACAO DE PAISES\n")
        f.write("=" * 80 + "\n")
        f.write(f"Data/Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Pais Atual: {current_country}\n")
        f.write(f"Pais Destino: {destination_country}\n")
        f.write(f"Limite de Iteracoes: {max_iterations}\n")
        f.write("=" * 80 + "\n\n")
        
        # Mostrar no console apenas informação básica
        print("=" * 80)
        print("AGENTE DE COMPARACAO DE PAISES")
        print("=" * 80)
        print(f"Pais Atual: {current_country}")
        print(f"Pais Destino: {destination_country}")
        print(f"Processando...")
        
        # Loop principal de iterações
        while i < max_iterations:
            i += 1
            
            # Log da iteração no arquivo
            f.write("\n" + "=" * 80 + "\n")
            f.write(f"ITERACAO: {i}\n")
            f.write("=" * 80 + "\n\n")
            
            # Obter resposta do modelo
            result = agent(next_prompt)
            
            # Salvar resultado completo no log
            f.write("RESPOSTA DO MODELO:\n")
            f.write("-" * 80 + "\n")
            f.write(result + "\n")
            f.write("-" * 80 + "\n\n")
            
            # Verificar se precisa executar alguma ferramenta
            if "PAUSE" in result and "Action" in result:
                # Extrair a ação e argumentos usando regex
                action = re.findall(r"Action:\s*([a-z_]+):\s*(.+)", result, re.IGNORECASE)
                
                if action:
                    chosen_tool = action[0][0].strip()
                    arg = action[0][1].strip()
                    
                    # Log da ferramenta escolhida
                    f.write(f"FERRAMENTA ESCOLHIDA: {chosen_tool}\n")
                    f.write(f"ARGUMENTOS: {arg}\n\n")
                    
                    # Verificar se a ferramenta existe
                    if chosen_tool in tools:
                        try:
                            # Executar a ferramenta apropriada
                            if chosen_tool == 'get_country_info':
                                # Ferramenta 1: Buscar informações do país
                                result_tool = get_country_info(arg)
                            
                            elif chosen_tool == 'convert_currency':
                                # Ferramenta 2: Conversão de moedas
                                # Separar os argumentos (formato: "BRL, JPY")
                                args = [a.strip() for a in arg.split(',')]
                                if len(args) == 2:
                                    result_tool = convert_currency(args[0], args[1])
                                else:
                                    result_tool = "ERROR: convert_currency requires 2 arguments (from_code, to_code)"
                            
                            elif chosen_tool == 'calculate_timezone_diff':
                                # Ferramenta 3: Calcular diferença de timezone
                                # Separar os argumentos (formato: "UTC-03:00, UTC+09:00")
                                args = [a.strip() for a in arg.split(',')]
                                if len(args) == 2:
                                    result_tool = calculate_timezone_diff(args[0], args[1])
                                else:
                                    result_tool = "ERROR: calculate_timezone_diff requires 2 arguments (tz_from, tz_to)"
                            
                            # Preparar próximo prompt com o resultado da ferramenta
                            next_prompt = f"Observation: {result_tool}"
                            
                            # Salvar resultado da ferramenta no log
                            f.write("RESULTADO DA FERRAMENTA:\n")
                            f.write("-" * 80 + "\n")
                            f.write(result_tool + "\n")
                            f.write("-" * 80 + "\n\n")
                        
                        except Exception as e:
                            # Tratar erro na execução da ferramenta
                            error_msg = f"Error executing {chosen_tool}: {str(e)}"
                            next_prompt = f"Observation: {error_msg}"
                            
                            # Salvar erro no log
                            f.write("ERRO NA EXECUCAO DA FERRAMENTA:\n")
                            f.write("-" * 80 + "\n")
                            f.write(error_msg + "\n")
                            f.write("-" * 80 + "\n\n")
                    
                    else:
                        # Ferramenta não encontrada
                        next_prompt = "Observation: tool not found"
                        f.write("ERRO: Ferramenta nao encontrada\n\n")
                    
                    # Continuar para próxima iteração
                    continue
            
            # Verificar se chegou na resposta final
            if "Answer:" in result or "Answer" in result:
                # Salvar indicação de conclusão no log
                f.write("\n" + "=" * 80 + "\n")
                f.write("RESPOSTA FINAL OBTIDA\n")
                f.write("=" * 80 + "\n\n")
                
                # Extrair apenas a parte da resposta final
                answer_match = re.search(r"Answer:?\s*(.+)", result, re.DOTALL | re.IGNORECASE)
                if answer_match:
                    final_result = answer_match.group(1).strip()
                else:
                    final_result = result
                
                f.write("RESPOSTA FINAL:\n")
                f.write("-" * 80 + "\n")
                f.write(final_result + "\n")
                f.write("-" * 80 + "\n")
                
                break
        
        # Verificar se atingiu o limite de iterações sem concluir
        if i >= max_iterations:
            f.write("\n" + "=" * 80 + "\n")
            f.write("LIMITE DE ITERACOES ATINGIDO\n")
            f.write("=" * 80 + "\n")
            final_result = f"Limite de {max_iterations} iteracoes atingido sem resposta final."
    
    # Mostrar no console apenas a resposta final
    print("\n" + "=" * 80)
    print("RESPOSTA FINAL")
    print("=" * 80)
    print(final_result)
    print("\n" + "=" * 80)
    print(f"Log completo salvo em: {log_file}")
    print("=" * 80)
    
    return final_result




In [ ]:
# ============================================================================
# EXEMPLO DE USO
# ============================================================================

current_country= input("Me diga o seu pais de origem:\t")
destination_country= input("Me diga um pais que gostaria de saber mais:\t")
query= input('Você gostaria de fazer uma pergunta especifica comparando o seu pais de origuem a outro? Se sim, pode perguntar aqui, \nSenão, aperte Enter!\t')

resultado_final = agent_loop(
    max_iterations=15,                             # max_iterations (1º argumento)
    system=system_prompt,                          # system (2º argumento)
    current_country=current_country,               # current_country (3º argumento)
    destination_country=destination_country,       # destination_country (4º argumento)
    query=query,                                   # query (5º argumento)
    log_file=f"log_{current_country}_{destination_country}.txt"  # log_file (nomeado)
)